In [1]:
from func import *

In [2]:
train_df = load_csvs_from_ftp_to_df(provided_local_dir="/")
train_clean = train_df.copy()

In [3]:
quality_summary_table = data_quality_summary(train_df)
quality_summary_table.sort_values(by=["missing_%"], ascending=False)

,column,dtype,n_unique,missing_%,num_missing
75,MiddleOrJuniorSchoolDistrict,float64,0,100.0,68493
29,FireplacesTotal,float64,0,100.0,68493
31,AboveGradeFinishedArea,float64,0,100.0,68493
34,TaxAnnualAmount,float64,0,100.0,68493
49,TaxYear,float64,0,100.0,68493
...,...,...,...,...,...
36,MlsStatus,object,1,0.0,0
72,PostalCode,object,1691,0.0,1
35,CountyOrParish,object,58,0.0,0
32,ListingKeyNumeric,int64,68458,0.0,0


# Remove col with missing rate over 80%

In [4]:
train_clean, col_drop_list = remove_by_missing_pct(train_clean,
                                                   default_threshold=0.8)

## Remove duplicate rows

In [5]:
train_clean = remove_duplicate(train_clean)

# Convert email into email domain

In [6]:
train_clean = convert_email_domains(train_clean,
                                    email_col_name="ListAgentEmail",
                                    domain_col_name="EmailDomain")


# Remove erroneous or non-economic transactions, remove the top 0.5% and bottom 0.5% of ClosePrice

In [7]:
train_clean = remove_extreme_rows(train_clean,
                                  price_col="ClosePrice",
                                  upper_bound_pct=0.995,
                                  lower_bound_pct=0.005)

# Remove non-positive and negative rows according to constraints

In [8]:
positive_col_list = ["BedroomsTotal",
                     "BathroomsTotalInteger",
                     "LotSizeAcres",
                     "LotSizeArea",
                     "LotSizeSquareFeet"]
non_negative_col_list = ["ParkingTotal"]

In [9]:
train_clean = remove_by_positive_or_non_negative_constraint(train_clean,
                                                            positive_col_list=positive_col_list,
                                                            non_negative_col_list=non_negative_col_list)

# Remove row that exceed the range for latitude, longitude

In [10]:
train_clean = remove_by_location(train_clean,
                                 lat_min=32.5,
                                 lat_max=42.0,
                                 lon_min=-124.5,
                                 lon_max=-114.0)

# For "Levels", rencode it

In [11]:
train_clean = recode_levels_df_apply(train_clean)

# For "Flooring", using binary flag, fill the na with false

In [12]:
train_clean = create_flag(train_clean,
                          target_col_list=["Flooring"])

C:\My_Programs\Python_Project\IDX_Intern\func.py:246: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[val + "YN"] = df[val + "YN"].fillna(False)
C:\My_Programs\Python_Project\IDX_Intern\func.py:246: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[val + "YN"] = df[val + "YN"].fillna(False)
C:\My_Programs\Python_Project\IDX_Intern\func.py:246: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.

# For "CoListOfficeName", "MLSAreaMajor", "BuyerOfficeAOR", "BuyerOfficeName", "EmailDomain", fill with mode by city

In [13]:
col_fill_by_col_ref_mode_list = ["CoListOfficeName",
                                 "MLSAreaMajor",
                                 "BuyerOfficeAOR",
                                 "BuyerOfficeName",
                                 "EmailDomain",
                                 "BuyerAgentAOR",
                                 "ListAgentAOR"]

In [14]:
train_clean = fill_na_with_mode(df=train_clean,
                                target_col_list=col_fill_by_col_ref_mode_list,
                                reference_col="City")


# For school-related variables, Nearest-neighbor assignment through longitude and latitude


In [15]:
col_fill_by_knn_mode_list = ["ElementarySchool",
                             "MiddleOrJuniorSchool",
                             "HighSchool",
                             "HighSchoolDistrict"]

In [16]:
train_clean, knn_model = knn_impute_latlon(train_clean,
                                            target_col_list=col_fill_by_knn_mode_list,
                                            ref_col_List=["Longitude", "Latitude"],
                                            k=3,
                                            model=None)


# For "YN" variables, fill the na with False

In [17]:
yn_col_list = ["AttachedGarageYN",
               "ViewYN",
               "NewConstructionYN",
               "PoolPrivateYN",
               "FireplaceYN"]

In [18]:
train_clean[yn_col_list] = train_clean[yn_col_list].fillna(False)

C:\Users\lee39\AppData\Local\Temp\ipykernel_25720\3008758576.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_clean[yn_col_list] = train_clean[yn_col_list].fillna(False)


# For other variables, clustering according to all numeric variables, then fill the na

In [19]:
quality_summary_table = data_quality_summary(train_clean)
col_need_to_fill_na = quality_summary_table[quality_summary_table["num_missing"] > 0]["column"]
reference_col_list = quality_summary_table[quality_summary_table["num_missing"] == 0]["column"].drop(columns="ClosePrice", errors="ignore")

In [ ]:
train_clean, clustering_model, scaler = fill_na_by_cluster(df=train_clean,
                                                        target_col_list=col_need_to_fill_na,
                                                        reference_col_list=reference_col_list,
                                                        method="k-means",
                                                        num_clusters=10,
                                                        random_state=42,
                                                        model=None,
                                                        scaler_method="robust",
                                                        scaler=None)


# Save the processed data and correspond mode

In [24]:
save_file(train_clean,
          save_name="processed",
          data_type="train",
          model_dict={"knn_model": knn_model,
                      "clustering_model": clustering_model,
                      "scaler": scaler})
